<a href="https://colab.research.google.com/github/ElioPikuli/Secure-SMS/blob/main/Secure_SMS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

An Application for secure SMS exchange: encr-decr with
Serpent in OFB mode, including DH key generation and
signature EC EL-GAMAL

# **IMPORTS AND INSTALLS**

In [1]:
!pip install ipywidgets

import os
import random
import hashlib
import base64
import ipywidgets as widgets
from IPython.display import display, clear_output

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 19.5 MB/s eta 0:00:00


# **Diffie-Hellman Key Exchange**

In [2]:
# --- Diffie-Hellman Functions ---
def generate_dh_keys(p, g):
    priv = random.randint(2, p - 2)
    pub = pow(g, priv, p)
    return priv, pub

def compute_shared_secret(their_pub, my_priv, p):
    return pow(their_pub, my_priv, p)

# **Serpent Encryption in OFB Mode**

In [3]:
import struct

# Constants
BLOCK_SIZE = 16  # Serpent uses 128-bit blocks

# Simplified 4-bit S-box (Not real Serpent S-box, for demo)
SBOX = [
    0xE, 0x4, 0xD, 0x1,
    0x2, 0xF, 0xB, 0x8,
    0x3, 0xA, 0x6, 0xC,
    0x5, 0x9, 0x0, 0x7
]

SBOX_INV = [SBOX.index(x) for x in range(16)]

# Padding (PKCS#7 style)
def pad(plaintext):
    if not isinstance(plaintext, bytes):
        plaintext = plaintext.encode('utf-8')
    pad_len = BLOCK_SIZE - len(plaintext) % BLOCK_SIZE
    if pad_len == 0:  # If already a multiple of BLOCK_SIZE, add a full block of padding
        pad_len = BLOCK_SIZE
    return plaintext + bytes([pad_len] * pad_len)

def unpad(padded):
    try:
        pad_len = padded[-1]
        # Check if padding is valid - all padding bytes should be the same
        if pad_len < 1 or pad_len > BLOCK_SIZE:
            raise ValueError("Invalid padding length.")

        # Verify all padding bytes are correct (PKCS#7)
        for i in range(1, pad_len + 1):
            if padded[-i] != pad_len:
                raise ValueError("Inconsistent padding bytes.")

        return padded[:-pad_len]
    except Exception as e:
        # For debugging
        print(f"Padding error details: {e}")
        print(f"Last byte (padding length): {padded[-1] if padded else 'empty'}")
        print(f"Padded data length: {len(padded)}")
        if len(padded) >= 5:
            print(f"Last 5 bytes: {[b for b in padded[-5:]]}")

        # In case of any padding error, try a more lenient approach
        # by just returning the data without removing padding
        return padded

# Substitute each nibble using S-box
def sbox_substitution(block, inverse=False):
    result = bytearray()
    table = SBOX_INV if inverse else SBOX
    for byte in block:
        high = table[byte >> 4]
        low = table[byte & 0xF]
        result.append((high << 4) | low)
    return bytes(result)

# Convert integer to bytes for key usage
def int_to_bytes(n, length=BLOCK_SIZE):
    return n.to_bytes(length, byteorder='big')

# Very basic key schedule (just repeat the key)
def expand_key(key, num_blocks):
    # Ensure key is in bytes format
    if isinstance(key, int):
        key = int_to_bytes(key)
    elif isinstance(key, str):
        key = key.encode('utf-8')

    return [key[i % len(key):i % len(key) + BLOCK_SIZE].ljust(BLOCK_SIZE, b'\0') for i in range(num_blocks)]

# Simplified "Serpent block encryption" (fake round structure)
def serpent_encrypt_block(block, key):
    # Ensure key is in bytes format
    if isinstance(key, int):
        key = int_to_bytes(key)

    # Ensure key is proper length
    if len(key) < BLOCK_SIZE:
        key = key.ljust(BLOCK_SIZE, b'\0')
    elif len(key) > BLOCK_SIZE:
        key = key[:BLOCK_SIZE]

    for _ in range(32):  # Serpent has 32 rounds, simplified here
        block = sbox_substitution(bytes(a ^ b for a, b in zip(block, key)))
    return block

def serpent_decrypt_block(block, key):
    # Ensure key is in bytes format
    if isinstance(key, int):
        key = int_to_bytes(key)

    # Ensure key is proper length
    if len(key) < BLOCK_SIZE:
        key = key.ljust(BLOCK_SIZE, b'\0')
    elif len(key) > BLOCK_SIZE:
        key = key[:BLOCK_SIZE]

    for _ in range(32):
        block = sbox_substitution(bytes(a ^ b for a, b in zip(block, key)), inverse=True)
    return block

# OFB Mode
def ofb_mode_encrypt(plaintext, key, iv):
    # Ensure plaintext and key are in proper format
    if isinstance(plaintext, str):
        plaintext = plaintext.encode('utf-8')

    if isinstance(key, int):
        key = int_to_bytes(key)

    # Ensure IV is in bytes format and of correct length
    if isinstance(iv, int):
        iv = int_to_bytes(iv)
    elif isinstance(iv, str):
        iv = iv.encode('utf-8')

    # Ensure IV is proper length
    if len(iv) < BLOCK_SIZE:
        iv = iv.ljust(BLOCK_SIZE, b'\0')
    elif len(iv) > BLOCK_SIZE:
        iv = iv[:BLOCK_SIZE]

    plaintext = pad(plaintext)
    ciphertext = bytearray()
    feedback = iv

    for i in range(0, len(plaintext), BLOCK_SIZE):
        block = plaintext[i:i+BLOCK_SIZE]
        feedback = serpent_encrypt_block(feedback, key)
        cipher_block = bytes(a ^ b for a, b in zip(block, feedback))
        ciphertext.extend(cipher_block)

    return bytes(ciphertext)

def ofb_mode_decrypt(ciphertext, key, iv):
    # Ensure ciphertext and key are in proper format
    if isinstance(ciphertext, str):
        try:
            ciphertext = base64.b64decode(ciphertext)
        except:
            ciphertext = ciphertext.encode('utf-8')

    if isinstance(key, int):
        key = int_to_bytes(key)

    # Ensure IV is in bytes format and of correct length
    if isinstance(iv, int):
        iv = int_to_bytes(iv)
    elif isinstance(iv, str):
        iv = iv.encode('utf-8')

    # Ensure IV is proper length
    if len(iv) < BLOCK_SIZE:
        iv = iv.ljust(BLOCK_SIZE, b'\0')
    elif len(iv) > BLOCK_SIZE:
        iv = iv[:BLOCK_SIZE]

    # In OFB mode, decryption is the same operation as encryption
    decrypted = bytearray()
    feedback = iv

    for i in range(0, len(ciphertext), BLOCK_SIZE):
        block = ciphertext[i:i+BLOCK_SIZE]
        feedback = serpent_encrypt_block(feedback, key)
        plain_block = bytes(a ^ b for a, b in zip(block, feedback))
        decrypted.extend(plain_block)

    # Apply unpadding
    try:
        return unpad(bytes(decrypted))
    except Exception as e:
        print(f"Warning: Error during unpadding: {e}")
        # Return the raw decrypted bytes if unpadding fails
        return bytes(decrypted)

# **EC ElGamal Digital Signature**

In [4]:
# --- EC ElGamal Signature (simple curve over F_p) ---
p_ec = 751
a = -1
b = 188
g = (0, 376)
n = 727

O = None  # Point at infinity

def inverse_mod(k, p):
    return pow(k, -1, p)

def point_add(P, Q):
    if P is None: return Q
    if Q is None: return P

    x1, y1 = P
    x2, y2 = Q

    if x1 == x2 and y1 == y2:  # Point doubling
        if y1 == 0:  # Point at infinity
            return None
        # Calculate lambda for point doubling
        l = ((3 * x1 * x1 + a) * inverse_mod(2 * y1, p_ec)) % p_ec
    else:  # Point addition
        if x1 == x2:  # Vertical line, result is point at infinity
            return None
        # Calculate lambda for point addition
        l = ((y2 - y1) * inverse_mod(x2 - x1, p_ec)) % p_ec

    # Calculate new point coordinates
    x3 = (l * l - x1 - x2) % p_ec
    y3 = (l * (x1 - x3) - y1) % p_ec

    return (x3, y3)

def scalar_mult(k, P):
    result = None
    addend = P

    while k:
        if k & 1:
            result = point_add(result, addend)
        addend = point_add(addend, addend)
        k >>= 1

    return result

def generate_keypair():
    priv = random.randint(1, n - 1)
    pub = scalar_mult(priv, g)
    return priv, pub

def hash_message(msg):
    if isinstance(msg, bytes):
        h = hashlib.sha256(msg).hexdigest()
    else:
        h = hashlib.sha256(msg.encode()).hexdigest()
    return int(h, 16) % n

def sign(msg, priv):
    h = hash_message(msg)
    while True:
        k = random.randint(1, n - 1)
        R = scalar_mult(k, g)

        if R is None:  # Check if result is point at infinity
            continue

        r = R[0] % n
        if r == 0:
            continue

        k_inv = inverse_mod(k, n)
        s = (k_inv * (h + priv * r)) % n
        if s != 0:
            break

    return (r, s)

def verify(msg, signature, pub):
    r, s = signature
    if not (1 <= r < n and 1 <= s < n):
        return False

    h = hash_message(msg)
    s_inv = inverse_mod(s, n)
    u1 = (h * s_inv) % n
    u2 = (r * s_inv) % n

    P1 = scalar_mult(u1, g)
    P2 = scalar_mult(u2, pub)

    if P1 is None:
        return False if P2 is not None else (r == 0)
    if P2 is None:
        return False

    P = point_add(P1, P2)

    if P is None:  # Result is point at infinity
        return False

    return (P[0] % n) == r

# **Keys**

In [5]:
# --- Initialize keys ---
p = 9973
g_dh = 5

# Consistent random seed for reproducibility in testing
random.seed(42)

# DH key exchange
alice_dh_priv, alice_dh_pub = generate_dh_keys(p, g_dh)
bob_dh_priv, bob_dh_pub = generate_dh_keys(p, g_dh)

# Shared key - needs to be in bytes format for encryption
alice_shared_key = compute_shared_secret(bob_dh_pub, alice_dh_priv, p)
bob_shared_key = compute_shared_secret(alice_dh_pub, bob_dh_priv, p)

# Generate a fixed IV for testing (in practice, should be random for each message)
# iv = os.urandom(BLOCK_SIZE)
iv = b'\x01\x02\x03\x04\x05\x06\x07\x08\x09\x0A\x0B\x0C\x0D\x0E\x0F\x10'

# Convert shared key to bytes for encryption
alice_shared_key_bytes = int_to_bytes(alice_shared_key)
bob_shared_key_bytes = int_to_bytes(bob_shared_key)

print(f"Alice's shared key: {alice_shared_key}")
print(f"Bob's shared key: {bob_shared_key}")

# Verify alice and bob compute the same shared key
assert alice_shared_key == bob_shared_key, "Shared keys don't match!"

# EC key pairs
alice_ec_priv, alice_ec_pub = generate_keypair()
bob_ec_priv, bob_ec_pub = generate_keypair()

Alice's shared key: 9244
Bob's shared key: 9244


# **UI**

In [6]:
# --- UI Widgets ---
alice_input = widgets.Text(
    description="Alice:",
    placeholder='Enter message...',
    layout=widgets.Layout(width='80%')
)

bob_input = widgets.Text(
    description="Bob:",
    placeholder='Enter message...',
    layout=widgets.Layout(width='80%')
)

output = widgets.Output()

# --- Handlers ---
def handle_alice_send(_):
    with output:
        clear_output()
        msg = alice_input.value
        if not msg:
            print("Please enter a message first.")
            return

        try:
            # Sign and encrypt
            signature = sign(msg, alice_ec_priv)
            cipher = ofb_mode_encrypt(msg, alice_shared_key_bytes, iv)
            # Bob decrypts
            decrypted = ofb_mode_decrypt(cipher, bob_shared_key_bytes, iv)
            try:
                decoded = decrypted.decode('utf-8')
            except UnicodeDecodeError:
                decoded = str(decrypted)
                print("Warning: Could not decode message as UTF-8")
            # Bob verifies
            valid = verify(decoded, signature, alice_ec_pub)

            # Update conversation history
            update_conversation(
                sender="Alice",
                message=msg,
                ciphertext=base64.b64encode(cipher).decode(),
                decrypted=decoded,
                signature_valid=valid
            )

            # Clear input box after sending
            alice_input.value = ''

            print("Alice sends:")
            print("Ciphertext:", base64.b64encode(cipher).decode())
            print("Signature:", signature)
            print("Bob receives:")
            print("Decrypted:", decoded)
            print("Signature valid:", valid)
        except Exception as e:
            import traceback
            print(f"Error: {str(e)}")
            print(traceback.format_exc())

def handle_bob_send(_):
    with output:
        clear_output()
        msg = bob_input.value
        if not msg:
            print("Please enter a message first.")
            return

        try:
            # Sign and encrypt
            signature = sign(msg, bob_ec_priv)
            cipher = ofb_mode_encrypt(msg, bob_shared_key_bytes, iv)
            # Alice decrypts
            decrypted = ofb_mode_decrypt(cipher, alice_shared_key_bytes, iv)
            try:
                decoded = decrypted.decode('utf-8')
            except UnicodeDecodeError:
                decoded = str(decrypted)
                print("Warning: Could not decode message as UTF-8")
            # Alice verifies
            valid = verify(decoded, signature, bob_ec_pub)

            # Update conversation history
            update_conversation(
                sender="Bob",
                message=msg,
                ciphertext=base64.b64encode(cipher).decode(),
                decrypted=decoded,
                signature_valid=valid
            )

            # Clear input box after sending
            bob_input.value = ''

            print("Bob sends:")
            print("Ciphertext:", base64.b64encode(cipher).decode())
            print("Signature:", signature)
            print("Alice receives:")
            print("Decrypted:", decoded)
            print("Signature valid:", valid)
        except Exception as e:
            import traceback
            print(f"Error: {str(e)}")
            print(traceback.format_exc())

# --- Buttons ---
alice_btn = widgets.Button(description="Send (Alice)")
bob_btn = widgets.Button(description="Send (Bob)")
alice_btn.on_click(handle_alice_send)
bob_btn.on_click(handle_bob_send)

# --- Display ---
display(widgets.VBox([
    alice_input,
    alice_btn,
    bob_input,
    bob_btn,
    output
]))

# Conversation history widget
conversation_history = widgets.Textarea(
    value='',
    placeholder='Conversation will appear here...',
    description='Chat:',
    disabled=False,
    layout=widgets.Layout(width='100%', height='300px')
)

# Display the conversation box at the top of the UI
display(conversation_history)

# Function to append new messages to the chat box
def update_conversation(sender, message, ciphertext, decrypted, signature_valid):
    new_entry = f"""{sender} sent: {message}
Ciphertext (Base64): {ciphertext}
Decrypted by recipient: {decrypted}
Signature valid: {signature_valid}

"""
    conversation_history.value += new_entry


Textarea(value='', description='Chat:', layout=Layout(height='300px', width='100%'), placeholder='Conversation…